# Connected agents

This is a feature in the Azure AI Foundry Agent Service that allows you to break large tasks into smaller, specialized roles without building a custom orchestrator or hardcoding routing logic.

This division of labor helps:
- Simplify complex workflows
- Improve agent performance and accuracy
- Make systems easier to maintain and extend over time

Using connected agents to automate workflows offers many benefits:
- **No custom orchestration required** - The main agent uses natural language to route tasks, eliminating the need for hardcoded logic.
- **Improved reliability and traceability** - The clear separation of responsibilities makes issues easier to debug since agents can be tested individually.
- **Flexible and extensible** - Add or swap agents without reworking the entire system or modifying the main agent.

# Agent responsibilities

**Main agent responsibilities**
The main agent acts as the orchestrator. It interprets the intent behind a request (the input) and determines which connected agent is best suited to handle it. Then aggregating or summarize results.

**Connected agent responsibilities**
Connected agents designed to focus on a single domain of responsibility. A connected agent is responsible for completing a specific action based on a clear prompt, using tools (if needed), and returning the results to the main agent.

Connected agents should be designed with a single responsibility in mind. This makes your system easier to debug, extend, and reuse.

In [4]:
import os

from dotenv import load_dotenv

from azure.ai.agents import AgentsClient
from azure.ai.agents.models import ConnectedAgentTool, MessageRole, ListSortOrder, ToolSet, FunctionTool
from azure.identity import DefaultAzureCredential

load_dotenv()

AGENT_PROJECT_ENDPOINT = os.getenv("AGENT_PROJECT_ENDPOINT")
AGENT_MODEL_DEPLOYMENT = os.getenv("AGENT_MODEL_DEPLOYMENT")

agents_client = AgentsClient(
    endpoint=AGENT_PROJECT_ENDPOINT,
    credential=DefaultAzureCredential(
        exclude_environment_credential=True,
        exclude_managed_identity_credential=True
    )
)

In [9]:
# Instructions for the primary agent
triage_agent_instructions = """
Triage the given ticket. Use the connected tools to determine the ticket's priority, 
which team it should be assigned to, and how much effort it may take.
"""

# Priority agent definition
priority_agent_name = "priority_agent"
priority_agent_instructions = """
Assess how urgent a ticket is based on its description.

Respond with one of the following levels:
- High: User-facing or blocking issues
- Medium: Time-sensitive but not breaking anything
- Low: Cosmetic or non-urgent tasks

Only output the urgency level and a very brief explanation.
"""

# Team agent definition
team_agent_name = "team_agent"
team_agent_instructions = """
Decide which team should own each ticket.

Choose from the following teams:
- Frontend
- Backend
- Infrastructure
- Marketing

Base your answer on the content of the ticket. Respond with the team name and a very brief explanation.
"""

# Effort agent definition
effort_agent_name = "effort_agent"
effort_agent_instructions = """
Estimate how much work each ticket will require.

Use the following scale:
- Small: Can be completed in a day
- Medium: 2-3 days of work
- Large: Multi-day or cross-team effort

Base your estimate on the complexity implied by the ticket. Respond with the effort level and a brief justification.
"""

# Create the priority agent on the Azure AI agent service
priority_agent = agents_client.create_agent(
    model=AGENT_MODEL_DEPLOYMENT,
    name=priority_agent_name,
    instructions=priority_agent_instructions
)
# Create a connected agent tool for the priority agent
priority_agent_tool = ConnectedAgentTool(
    id=priority_agent.id, 
    name=priority_agent_name, 
    description="Assess the priority of a ticket"
)

# Create the team agent and connected tool
team_agent = agents_client.create_agent(
    model=AGENT_MODEL_DEPLOYMENT,
    name=team_agent_name,
    instructions=team_agent_instructions
)
team_agent_tool = ConnectedAgentTool(
    id=team_agent.id, 
    name=team_agent_name, 
    description="Determines which team should take the ticket"
)

# Create the effort agent and connected tool
effort_agent = agents_client.create_agent(
    model=AGENT_MODEL_DEPLOYMENT,
    name=effort_agent_name,
    instructions=effort_agent_instructions
)
effort_agent_tool = ConnectedAgentTool(
    id=effort_agent.id, 
    name=effort_agent_name, 
    description="Determines the effort required to complete the ticket"
)

# Create a main agent with the Connected Agent tools
agent = agents_client.create_agent(
    model=AGENT_MODEL_DEPLOYMENT,
    name="triage-agent",
    instructions=triage_agent_instructions,
    tools=[
        priority_agent_tool.definitions[0],
        team_agent_tool.definitions[0],
        effort_agent_tool.definitions[0]
    ]
)

In [10]:
# Create thread for the chat session
print("Creating agent thread.")
thread = agents_client.threads.create()

Creating agent thread.


In [11]:
# Create the ticket prompt
prompt = "Investigate occasional 502 errors from the search endpoint.”"
# Send a prompt to the agent
message = agents_client.messages.create(
    thread_id=thread.id,
    role=MessageRole.USER,
    content=prompt,
)
# Create and process Agent run in thread with tools
print("Processing agent thread. Please wait.")
run = agents_client.runs.create_and_process(thread_id=thread.id, agent_id=agent.id)

Processing agent thread. Please wait.


In [12]:
# Fetch and log all messages
messages = agents_client.messages.list(thread_id=thread.id, order=ListSortOrder.ASCENDING)
for message in messages:
    if message.text_messages:
        last_msg = message.text_messages[-1]
        print(f"{message.role}:\n{last_msg.text.value}\n")

# Delete the agent when done
print("Cleaning up agents:")
agents_client.delete_agent(agent.id)
print("Deleted triage agent.")

# Delete the connected agents when done
agents_client.delete_agent(priority_agent.id)
print("Deleted priority agent.")
agents_client.delete_agent(team_agent.id)
print("Deleted team agent.")
agents_client.delete_agent(effort_agent.id)
print("Deleted effort agent.")

MessageRole.USER:
Investigate occasional 502 errors from the search endpoint.”

MessageRole.AGENT:
Here's the triage for the ticket regarding the occasional 502 errors from the search endpoint:

- **Priority**: High. This issue is user-facing and could block access to crucial functionality .
- **Assigned Team**: Backend. The issue is related to server-side errors, which fall under the purview of the backend team .
- **Effort Level**: Large. Investigating 502 errors typically requires analyzing server configurations, load balancing, and interactions between services. This process may involve collaboration across multiple teams and could extend over several days .

Cleaning up agents:
Deleted triage agent.
Deleted priority agent.
Deleted team agent.
Deleted effort agent.
